In [1]:
import pandas as pd     #(version 1.0.0)
import plotly           #(version 4.5.4) pip install plotly==4.5.4
import plotly.express as px

import dash             #(version 1.9.1) pip install dash==1.9.1
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import requests
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mlxtend.plotting import plot_pca_correlation_graph
import numpy as np
import plotly.express as px
from scipy import stats
from scipy.stats import zscore
import requests
import seaborn as sns
from matplotlib import colors as mcolors
from sklearn.manifold import TSNE
import warnings
import matplotlib.cbook
from sklearn.svm import SVC
# import umap.umap_ as UMAP

warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

In [2]:
import types

def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

list(imports())

['builtins',
 'builtins',
 'pandas',
 'plotly',
 'plotly.express',
 'dash',
 'dash_core_components',
 'dash_html_components',
 'requests',
 'matplotlib.pyplot',
 'numpy',
 'scipy.stats',
 'seaborn',
 'matplotlib.colors',
 'warnings',
 'matplotlib',
 'types']

In [3]:
!pip freeze

certifi==2020.12.5
chardet==4.0.0
docopt==0.6.2
idna==2.10
numpy==1.20.2
pipreqs==0.4.10
requests==2.25.1
urllib3==1.26.4
yarg==0.1.9


In [2]:
# read each dataset to dataframe
#Individuals 16 years and older actively seeking employment (within the last four weeks) who are unemployed.
Unemployment = pd.read_csv('./Eco_Data/Unemployment Rate .csv', skiprows= [0,1,2,3,4])
Unemployment = Unemployment.rename({'Data': 'Unemployment_Rate'}, axis=1)
#Individuals with incomes below the Federal Poverty Level (FPL
Poverty = pd.read_csv('./Eco_Data/Poverty.csv', skiprows= [0,1,2,3,4])
Poverty = Poverty.rename({'Data': 'Poverty'}, axis=1)
#ratio represents the proportion of the population aged 16 and over that are currently employed
Employment_Population = pd.read_csv('./Eco_Data/Employment Population Ratio.csv', skiprows= [0,1,2,3,4])
Employment_Population = Employment_Population.rename({'Data': 'Employment_Population'}, axis=1)
#Individuals without health insurance
Uninsured = pd.read_csv('./Health/Uninsured.csv', skiprows= [0,1,2,3,4])
Uninsured = Uninsured.rename({'Data': 'Uninsured'}, axis=1)
#Family Income
Family_Income = pd.read_csv('./Eco_Data/Family Income.csv', skiprows= [0,1,2,3,4])
Family_Income = Family_Income.rename({'Data': 'Family_Income'}, axis=1)
#Educational attainment for adults ages 25 and over
Educational_Attainment = pd.read_csv('./Eco_Data/Educational Attainment.csv', skiprows= [0,1,2,3,4])
Educational_Attainment = Educational_Attainment.rename({'Data': 'Educational_Attainment'}, axis=1)



In [3]:
Unemployment = Unemployment[(Unemployment['Location'] != 'New York City' )]
Poverty = Poverty[(Poverty['Location'] != 'New York City' )]
Employment_Population = Employment_Population[(Employment_Population['Location'] != 'New York City' )]
Uninsured = Uninsured[(Uninsured['Location'] != 'New York City' )]
Family_Income = Family_Income[(Family_Income['Location'] != 'New York City' )]
Educational_Attainment = Educational_Attainment[(Educational_Attainment['Location'] != 'New York City' )]

# find the common colunm and get the proper infomation 
Poverty = Poverty[(Poverty['DataFormat'] == 'Percent' )]

# All age
Uninsured = Uninsured[
                      (Uninsured['DataFormat'] == 'Percent' )]
# Income Level under $15,000
Family_Income = Family_Income[
                             (Family_Income['DataFormat'] == 'Percent' )]
# Less than High School Degree 
Educational_Attainment = Educational_Attainment[
                                                (Educational_Attainment['DataFormat'] == 'Percent' )]

# ['Age Group'] == 'All Ages' )& ['Income Level'] == 'Under $15,000' ) & ['Education Level'] == 
#                                                  'Less than High School Degree') &
# merge the data to one dataframe
df = pd.merge(Unemployment, Poverty,on = [ "TimeFrame","Fips","Location"] )
df = pd.merge(df, Employment_Population,on = [ "TimeFrame","Fips","Location"] )
df = pd.merge(df, Uninsured,on = [ "TimeFrame","Fips","Location"] )
df = pd.merge(df, Family_Income,on = [ "TimeFrame","Fips","Location"] )
df = pd.merge(df, Educational_Attainment,on = [ "TimeFrame","Fips","Location"] )
def remove_boro(df):
    df = df[(df['Location'] != 'Bronx' ) &
                            (df['Location'] != 'Brooklyn' )&
                            (df['Location'] != 'Manhattan' )&
                            (df['Location'] != 'Queens')& 
                            (df['Location'] != 'Staten Island')]
    return df
df = remove_boro(df)
Unemployment = remove_boro(Unemployment)
Poverty = remove_boro(Poverty)
Employment_Population = remove_boro(Employment_Population)
Uninsured = remove_boro(Uninsured)
Family_Income = remove_boro(Family_Income)
Educational_Attainment = remove_boro(Educational_Attainment)
# Unemployment, Poverty, and Employment_Population
df_upe = pd.merge(Unemployment, Poverty,on = [ "TimeFrame","Fips","Location"] )
df_upe = pd.merge(df_upe, Employment_Population,on = [ "TimeFrame","Fips","Location"] )
# df_upe['Unemployment_Rate'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in df_upe['Unemployment_Rate']], index = df_upe.index)
# df_upe['Poverty'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in df_upe['Poverty']], index = df_upe.index)
# df_upe['Employment_Population'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in df_upe['Employment_Population']], index = df_upe.index)
# Uninsured['Uninsured'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in Uninsured['Uninsured']], index = Uninsured.index)
# Family_Income['Family_Income'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in Family_Income['Family_Income']], index = Family_Income.index)
# Educational_Attainment['Educational_Attainment'] = pd.Series(["{0:.2f}%".format(val * 100) 
#                                     for val in Educational_Attainment['Educational_Attainment']], index = Educational_Attainment.index)
map_df = df.copy(deep = True)

In [4]:
df_upe.head()

,Location,TimeFrame,DataFormat_x,Unemployment_Rate,Fips,DataFormat_y,Poverty,DataFormat,Employment_Population
0,Mott Haven,2005,Percent,0.176,201,Percent,0.52191,Percent,0.391
1,Hunts Point,2005,Percent,0.176,202,Percent,0.52191,Percent,0.391
2,Morrisania,2005,Percent,0.165,203,Percent,0.42565,Percent,0.447
3,East Tremont,2005,Percent,0.165,206,Percent,0.42565,Percent,0.447
4,Bedford Stuyvesant,2005,Percent,0.139,303,Percent,0.26670,Percent,0.482


In [5]:
#Aplly Z-score

df1 = df
df1 = df1[df1.columns[df1.columns.isin(['Unemployment_Rate', 'Poverty','Employment_Population','Uninsured',
                                        'Family_Income','Educational_Attainment'])]]

df1 = df1.dropna(how='any') # drop nan entries
df1 = df1[(np.abs(stats.zscore(df1)) < 3).all(axis=1)] # remove outliers
df1 = df1.apply(zscore) # Normalization
df['Unemployment_Rate'] = df1['Unemployment_Rate']
df['Poverty'] = df1['Poverty']
df['Employment_Population'] = df1['Employment_Population']
df['Uninsured'] = df1['Uninsured']
df['Family_Income'] = df1['Family_Income']
df['Educational_Attainment'] = df1['Educational_Attainment']




In [6]:
def covert_df(df):
    df_test = df

    # 0-11 bronx
    # 12-23 Manhattan
    # 24-26 Staten Island
    # 27-44 Brooklyn
    # 45-58 Queens

    CD_Name = ['Riverdale', 'Williamsbridge', 'Throgs Neck', 'Pelham Parkway1',
               'Morrisania', 'East Tremont', 'Bedford Park', 'University Heights', 
               'Concourse/Highbridge', 'Unionport/Soundview', 'Mott Haven', 'Hunts Point', 
               'Washington Heights', 'Manhattanville', 'Central Harlem', 'East Harlem', 
               'Upper East Side', 'Upper West Side', 'Chelsea/Clinton', 'Midtown Business District', 
               'Murray Hill/Stuyvesant', 'Lower East Side', 'Battery Park/Tribeca', 'Greenwich Village',
               'Tottenville', 'South Beach', 'St. George', 'Williamsburg/Greenpoint', 'Bushwick', 
               'Bedford Stuyvesant', 'Fort Greene/Brooklyn Hts', 'Park Slope', 'Crown Heights North',
               'Brownsville', 'East New York', 'Canarsie', 'East Flatbush', 'Crown Heights South', 
               'Sunset Park', 'Bay Ridge', 'Borough Park', 'Flatbush/Midwood', 'Sheepshead Bay',
               'Bensonhurst', 'Coney Island', 'Astoria', 'Jackson Heights', 'Flushing', 'Bayside',
               'Queens Village', 'Fresh Meadows/Briarwood', 'Elmhurst/Corona', 'Rego Park/Forest Hills',
               'Sunnyside/Woodside', 'Ridgewood/Glendale', 'Woodhaven', 'Jamaica/St. Albans', 
               'Howard Beach', 'The Rockaways']
    
    Borough = list()

    for i in range(59):
        if i < 12:
            Borough.append('Bronx')
        elif i > 11 and i < 24:
             Borough.append('Manhattan')
        elif i > 23 and i < 27:
             Borough.append('Staten Island')
        elif i > 26 and i < 45:
             Borough.append('Brooklyn')
        else:
            Borough.append('Queens')

    Location_ = df_test['Location'].values.tolist()
    len_L = len(Location_)
    CD_Boro = {CD_Name[i]: Borough[i] for i in range(len(CD_Name))}
    Get_Borough = [None] * len_L
    for x in range(len_L):
        for i, n in CD_Boro.items():
            if Location_[x] in i:
                Get_Borough[x] = n



    df_test['Borough'] = Get_Borough

    df = df_test

    cd =[ 'Astoria','Q01','Battery Park/Tribeca', 'M01','Bay Ridge','K10','Bayside','Q11',
    'Bedford Park','B07','Bedford Stuyvesant' ,'K03','Bensonhurst','K11','Borough Park','K12',
    'Brownsville','K16','Bushwick','K04', 'Canarsie','K18','Central Harlem','M10',
    'Chelsea/Clinton','M04','Concourse/Highbridge','B04','Coney Island','K13','Crown Heights North','K08',
    'Crown Heights South', 'K09', 'East Flatbush', 'K17','East Harlem', 'M11','East New York','K05',
    'East Tremont', 'B06','Elmhurst/Corona', 'Q04','Flatbush/Midwood', 'K14', 'Flushing', 'Q07', 
    'Fort Greene/Brooklyn Hts', 'K02','Fresh Meadows/Briarwood', 'Q08','Greenwich Village', 'M02', 'Howard Beach',
    'Q10', 'Hunts Point', 'B02', 'Jackson Heights', 'Q03', 'Jamaica/St. Albans','Q12', 'Lower East Side', 'M03',
    'Manhattanville', 'M09','Midtown Business District', 'M05', 'Morrisania', 'B03', 'Mott Haven', 'B01', 
    'Murray Hill/Stuyvesant', 'M06','Park Slope', 'K06', 'Pelham Parkway', 'B11', 'Queens Village', 
    'Q13', 'Rego Park/Forest Hills', 'Q06','Ridgewood/Glendale', 'Q05', 'Riverdale', 'B08', 'Sheepshead Bay', 
    'K15', 'South Beach','S02','St. George', 'S01', 'Sunnyside/Woodside', 'Q02', 'Sunset Park','K07', 
    'The Rockaways', 'Q14','Throgs Neck', 'B10', 'Tottenville', 'S03', 'Unionport/Soundview', 'B09', 
    'University Heights', 'B05','Upper East Side', 'M08', 'Upper West Side', 'M07', 'Washington Heights', 
    'M12', 'Williamsbridge', 'B12','Williamsburg/Greenpoint', 'K01', 'Woodhaven', 'Q09']
    CD_Name = list()
    CD_Num = list()
    i = 0
    while i < len(cd):
        CD_Name.append(cd[i])
        i = i + 1
        CD_Num.append(cd[i])
        i += 1
    cd = dict(zip(CD_Name, CD_Num))
    df['Boro_CD'] = df['Location'].map(cd)
    df['Boro_CD'] = df['Boro_CD'].str.replace('B','2')
    df['Boro_CD'] = df['Boro_CD'].str.replace('K','3')
    df['Boro_CD'] = df['Boro_CD'].str.replace('M','1')
    df['Boro_CD'] = df['Boro_CD'].str.replace('Q','4')
    df['Boro_CD'] = df['Boro_CD'].str.replace('S','5')
    df = df.dropna(how='any')

    return df

df = covert_df(df)
map_df = covert_df(map_df)


In [7]:
df.head()

,Location,TimeFrame,DataFormat_x,Unemployment_Rate,Fips,DataFormat_y,Poverty,DataFormat_x,Employment_Population,Age Group,DataFormat_y,Uninsured,Income Level,DataFormat_x,Family_Income,Education Level,DataFormat_y,Educational_Attainment,Borough,Boro_CD
600,Riverdale,2008,Percent,-0.607042,208,Percent,-0.525819,Percent,0.189437,Adults 19 Years and Older,Percent,1.244061,"Under $15,000",Percent,-0.083553,Less than High School Degree,Percent,0.011434,Bronx,208
601,Riverdale,2008,Percent,-0.607042,208,Percent,-0.525819,Percent,0.189437,Adults 19 Years and Older,Percent,1.244061,"Under $15,000",Percent,-0.083553,High School Degree,Percent,0.028562,Bronx,208
602,Riverdale,2008,Percent,-0.607042,208,Percent,-0.525819,Percent,0.189437,Adults 19 Years and Older,Percent,1.244061,"Under $15,000",Percent,-0.083553,Some College,Percent,-0.256558,Bronx,208
603,Riverdale,2008,Percent,-0.607042,208,Percent,-0.525819,Percent,0.189437,Adults 19 Years and Older,Percent,1.244061,"Under $15,000",Percent,-0.083553,Associate's Degree,Percent,-1.174813,Bronx,208
604,Riverdale,2008,Percent,-0.607042,208,Percent,-0.525819,Percent,0.189437,Adults 19 Years and Older,Percent,1.244061,"Under $15,000",Percent,-0.083553,Bachelor's Degree or Higher,Percent,2.040997,Bronx,208


In [8]:

r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
r = r.json()

In [9]:
def select_ML(df,ML):
        features = ['Unemployment_Rate','Poverty','Employment_Population','Uninsured',
                   'Family_Income','Educational_Attainment']
   
        
        # Set up the target or y for LDA training 
        # y = df.loc[:,['TimeFrame']].values # change to location
        y = df.loc[:,['Borough']].values # change to location
     
        # Target name
        # targets = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
        targets = ['Bronx','Brooklyn' ,'Manhattan' ,'Queens', 'Staten Island']
#         targets = Location_name
        for n in range(0,len(y)):
            for i in range(len(targets)):
                if y[n] == targets[i]:
                    y[n] = targets.index(targets[i])

        
        
        # set to contiguous flattened array
        y = y.ravel()
        # set the data type to integer
        y = y.astype('int')
      
        # Separate the features from the dataset
        X = df.loc[:,features].values
        trainX = df.loc[:,features].values
       

        # Principal component analysis
        pca = PCA(n_components=2)
        pca_components = pca.fit(X).transform(X)
        pca_components_df = pd.DataFrame(pca_components, columns = ['x','y'])
        pca_components_df["Location"] = df["Location"].tolist()
        
        lda = LinearDiscriminantAnalysis(n_components=2)
        lda_components = lda.fit(X, y).transform(X)
        lda_components_df = pd.DataFrame(lda_components, columns = ['x','y'])
        lda_components_df["Location"] = df["Location"].tolist()
      
        tsne = TSNE(n_components=2, random_state=0)
        tsne_components = tsne.fit_transform(X)
        tsne_components_df = pd.DataFrame(tsne_components, columns = ['x','y'])
        tsne_components_df["Location"] = df["Location"].tolist()

      
    
#         fig =  px.scatter(df_2019, x='x', y='y', color=df_2019['Borough'], hover_name="Location")
        

        pca_fig = px.scatter(pca_components_df, x='x', y='y', color=df['Borough'],hover_name="Location",labels={'0': 'PC 1', '1': 'PC 2'},title = "PCA")
        lda_fig = px.scatter(lda_components_df, x='x', y='y', color=df['Borough'],hover_name="Location",labels={'0': 'LD 1', '1': 'LD 2'},title = "LDA")
        tsne_fig = px.scatter(tsne_components_df, x='x', y='y', color=df['Borough'],hover_name="Location",labels={'0': 'TSNE 1', '1': 'TSNE 2'},title = 'Tsne')
        
        if ML == "pca":
            return pca_fig
        elif ML == 'lda':
            return lda_fig
        elif ML == "tsne":
            return tsne_fig
        else:
            return "can't find ML"
        

In [10]:

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div( children = [
    # All elements from the top of the page
    html.Div([
        dcc.Graph(id='graph-with-slider'),
        dcc.Dropdown(id='income_level_of_family_income',
            options=[
                     {'label': 'Under $15,000', 'value': 'Under $15,000'},
                     {'label': '$15,000 to $24,999', 'value': '$15,000 to $24,999'},
                     {'label': '$25,000 to $34,999', 'value': '$25,000 to $34,999'},
                     {'label': '$50,000 to $74,999', 'value': '$50,000 to $74,999'},
                     {'label': '$75,000 to $99,999', 'value': '$75,000 to $99,999'},
                     {'label': '$100,000 to $199,999', 'value': '$100,000 to $199,999'},
                     {'label': '$200,000 or more', 'value': '$200,000 or more'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Under $15,000',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='education',
            options=[
                     {'label': 'Less than High School Degree', 'value': 'Less than High School Degree'},
                     {'label': 'High School Degree', 'value': 'High School Degree'},
                     {'label': 'Some College', 'value': 'Some College'},
                     {'label': "Associate's Degree", 'value': "Associate's Degree"},
                     {'label': "Bachelor's Degree or Higher", 'value': "Bachelor's Degree or Higher"},
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value="Less than High School Degree",          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='age_Group_of_Uninsured',
            options=[
                     {'label': 'All Ages', 'value': 'All Ages'},
                     {'label': 'Adults 19 Years and Older', 'value': 'Adults 19 Years and Older'},  
                     {'label': 'Children Under 19 Years', 'value': 'Children Under 19 Years'}
             
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='All Ages',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Slider(
            id='year-slider',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )
    ]),
    # New Div for all elements in the new 'row' of the page
    html.Div([
        dcc.Graph(id='graph-with-slider2'),
        dcc.Dropdown(id='income_level_of_family_income2',
            options=[
                     {'label': 'Under $15,000', 'value': 'Under $15,000'},
                     {'label': '$15,000 to $24,999', 'value': '$15,000 to $24,999'},
                     {'label': '$25,000 to $34,999', 'value': '$25,000 to $34,999'},
                     {'label': '$50,000 to $74,999', 'value': '$50,000 to $74,999'},
                     {'label': '$75,000 to $99,999', 'value': '$75,000 to $99,999'},
                     {'label': '$100,000 to $199,999', 'value': '$100,000 to $199,999'},
                     {'label': '$200,000 or more', 'value': '$200,000 or more'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Under $15,000',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='education2',
            options=[
                     {'label': 'Less than High School Degree', 'value': 'Less than High School Degree'},
                     {'label': 'High School Degree', 'value': 'High School Degree'},
                     {'label': 'Some College', 'value': 'Some College'},
                     {'label': "Associate's Degree", 'value': "Associate's Degree"},
                     {'label': "Bachelor's Degree or Higher", 'value': "Bachelor's Degree or Higher"},
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value="Less than High School Degree",          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='age_Group_of_Uninsured2',
            options=[
                     {'label': 'All Ages', 'value': 'All Ages'},
                     {'label': 'Adults 19 Years and Older', 'value': 'Adults 19 Years and Older'},  
                     {'label': 'Children Under 19 Years', 'value': 'Children Under 19 Years'}
             
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='All Ages',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Slider(
            id='year-slider2',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )
    ]),

    html.Div([
        dcc.Graph(id='graph-with-slider3'),
        dcc.Dropdown(id='income_level_of_family_income3',
            options=[
                     {'label': 'Under $15,000', 'value': 'Under $15,000'},
                     {'label': '$15,000 to $24,999', 'value': '$15,000 to $24,999'},
                     {'label': '$25,000 to $34,999', 'value': '$25,000 to $34,999'},
                     {'label': '$50,000 to $74,999', 'value': '$50,000 to $74,999'},
                     {'label': '$75,000 to $99,999', 'value': '$75,000 to $99,999'},
                     {'label': '$100,000 to $199,999', 'value': '$100,000 to $199,999'},
                     {'label': '$200,000 or more', 'value': '$200,000 or more'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Under $15,000',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='education3',
            options=[
                     {'label': 'Less than High School Degree', 'value': 'Less than High School Degree'},
                     {'label': 'High School Degree', 'value': 'High School Degree'},
                     {'label': 'Some College', 'value': 'Some College'},
                     {'label': "Associate's Degree", 'value': "Associate's Degree"},
                     {'label': "Bachelor's Degree or Higher", 'value': "Bachelor's Degree or Higher"},
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value="Less than High School Degree",          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='age_Group_of_Uninsured3',
            options=[
                     {'label': 'All Ages', 'value': 'All Ages'},
                     {'label': 'Adults 19 Years and Older', 'value': 'Adults 19 Years and Older'},  
                     {'label': 'Children Under 19 Years', 'value': 'Children Under 19 Years'}
             
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='All Ages',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Slider(
            id='year-slider3',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )
    ]),
        html.Div([

        dcc.Graph(id='graph-with-slider4'),
        dcc.Dropdown(id='my_dropdown',
            options=[
                     {'label': 'Unemployment Rate', 'value': 'Unemployment_Rate'},
                     {'label': 'Poverty', 'value': 'Poverty'},
                     {'label': 'Employment Population', 'value': 'Employment_Population'},
                     {'label': 'Uninsured', 'value': 'Uninsured'},
                     {'label': 'Family Income', 'value': 'Family_Income'},
                     {'label': 'Educational Attainment', 'value': 'Educational_Attainment'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Unemployment_Rate',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),
        dcc.Dropdown(id='income_level_of_family_income4',
            options=[
                     {'label': 'Under $15,000', 'value': 'Under $15,000'},
                     {'label': '$15,000 to $24,999', 'value': '$15,000 to $24,999'},
                     {'label': '$25,000 to $34,999', 'value': '$25,000 to $34,999'},
                     {'label': '$50,000 to $74,999', 'value': '$50,000 to $74,999'},
                     {'label': '$75,000 to $99,999', 'value': '$75,000 to $99,999'},
                     {'label': '$100,000 to $199,999', 'value': '$100,000 to $199,999'},
                     {'label': '$200,000 or more', 'value': '$200,000 or more'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Under $15,000',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='education4',
            options=[
                     {'label': 'Less than High School Degree', 'value': 'Less than High School Degree'},
                     {'label': 'High School Degree', 'value': 'High School Degree'},
                     {'label': 'Some College', 'value': 'Some College'},
                     {'label': "Associate's Degree", 'value': "Associate's Degree"},
                     {'label': "Bachelor's Degree or Higher", 'value': "Bachelor's Degree or Higher"},
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value="Less than High School Degree",          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Dropdown(id='age_Group_of_Uninsured4',
            options=[
                     {'label': 'All Ages', 'value': 'All Ages'},
                     {'label': 'Adults 19 Years and Older', 'value': 'Adults 19 Years and Older'},  
                     {'label': 'Children Under 19 Years', 'value': 'Children Under 19 Years'}
             
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='All Ages',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Slider(
            id='year-slider4',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )

       
        
    ]),
     html.Div([

        dcc.Graph(id='graph-with-slider5'),
        dcc.Dropdown(id='my_dropdown1',
            options=[
                     {'label': 'Unemployment Rate', 'value': 'Unemployment_Rate'},
                     {'label': 'Poverty', 'value': 'Poverty'},
                     {'label': 'Employment Population', 'value': 'Employment_Population'},
               
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Unemployment_Rate',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),
        
        
       
        dcc.Slider(
            id='year-slider5',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )

       
        
    ]),
    html.Div([

        dcc.Graph(id='graph-with-slider6'),
        dcc.Dropdown(id='my_dropdown6',
            options=[
                   
        
             
                     {'label': 'Family Income', 'value': 'Family_Income'},
                  
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Family_Income',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),
        dcc.Dropdown(id='income_level_of_family_income6',
            options=[
                     {'label': 'Under $15,000', 'value': 'Under $15,000'},
                     {'label': '$15,000 to $24,999', 'value': '$15,000 to $24,999'},
                     {'label': '$25,000 to $34,999', 'value': '$25,000 to $34,999'},
                     {'label': '$50,000 to $74,999', 'value': '$50,000 to $74,999'},
                     {'label': '$75,000 to $99,999', 'value': '$75,000 to $99,999'},
                     {'label': '$100,000 to $199,999', 'value': '$100,000 to $199,999'},
                     {'label': '$200,000 or more', 'value': '$200,000 or more'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Under $15,000',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),

    
        dcc.Slider(
            id='year-slider6',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )

       
        
    ]),
    html.Div([

        dcc.Graph(id='graph-with-slider7'),
        dcc.Dropdown(id='my_dropdown7',
            options=[
                   
                     {'label': 'Educational Attainment', 'value': 'Educational_Attainment'}
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Educational_Attainment',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),
    
        dcc.Dropdown(id='education7',
            options=[
                     {'label': 'Less than High School Degree', 'value': 'Less than High School Degree'},
                     {'label': 'High School Degree', 'value': 'High School Degree'},
                     {'label': 'Some College', 'value': 'Some College'},
                     {'label': "Associate's Degree", 'value': "Associate's Degree"},
                     {'label': "Bachelor's Degree or Higher", 'value': "Bachelor's Degree or Higher"},
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value="Less than High School Degree",          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
      
        dcc.Slider(
            id='year-slider7',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )

       
        
    ]),
       html.Div([

        dcc.Graph(id='graph-with-slider8'),
        dcc.Dropdown(id='my_dropdown8',
            options=[
                     {'label': 'Uninsured', 'value': 'Uninsured'},
            
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='Uninsured',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"100%"},             #use dictionary to define CSS styles of your dropdown
            # className='select_box',           #activate separate CSS document in assets folder
            # persistence=True,                 #remembers dropdown value. Used with persistence_type
            # persistence_type='memory'         #remembers dropdown value selected until...
            ),

        dcc.Dropdown(id='age_Group_of_Uninsured8',
            options=[
                     {'label': 'All Ages', 'value': 'All Ages'},
                     {'label': 'Adults 19 Years and Older', 'value': 'Adults 19 Years and Older'},  
                     {'label': 'Children Under 19 Years', 'value': 'Children Under 19 Years'}
             
                 
                   
                   
            ],
            optionHeight=35,                    #height/space between dropdown options
            value='All Ages',          #dropdown value selected automatically when page loads
            disabled=False,                     #disable dropdown value selection
            multi=False,                        #allow multiple dropdown values to be selected
            searchable=True,                    #allow user-searching of dropdown values
            search_value='',                    #remembers the value searched in dropdown
            placeholder='Please select...',     #gray, default text shown when no option is selected
            clearable=True,                     #allow user to removes the selected value
            style={'width':"50%"},             #use dictionary to define CSS styles of your dropdown
          
            ),
        dcc.Slider(
            id='year-slider8',
            min=df['TimeFrame'].min(),
            max=df['TimeFrame'].max(),
            value=df['TimeFrame'].min(),
            marks={str(year): str(year) for year in df['TimeFrame'].unique()},
            step=None
        )

       
        
    ]),
])






In [ ]:
@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('year-slider', 'value'),
    Input('income_level_of_family_income', 'value'),
    Input('education', 'value'),
    Input('age_Group_of_Uninsured', 'value'))


def update_figure1(selected_year,income_level_of_family_income,education,age_Group_of_Uninsured):
    
    
    df_sel = df[(df['TimeFrame'] == int(selected_year)) & (df['Income Level'] == income_level_of_family_income) 
                & (df['Education Level'] == education ) & (df['Age Group'] == age_Group_of_Uninsured ) 
               ]


    fig = select_ML(df_sel,'pca')
   
    
#     fig.update_yaxes(
#     scaleanchor = "x",
#     scaleratio = 1,
        
#       )
    
    return fig
@app.callback(
    Output('graph-with-slider2', 'figure'),
    Input('year-slider2', 'value'),
    Input('income_level_of_family_income2', 'value'),
    Input('education2', 'value'),
    Input('age_Group_of_Uninsured2', 'value'))
def update_figure2(selected_year,income_level_of_family_income,education,age_Group_of_Uninsured):
    
   
    df_sel = df[(df['TimeFrame'] == int(selected_year)) & (df['Income Level'] == income_level_of_family_income) 
                & (df['Education Level'] == education ) & (df['Age Group'] == age_Group_of_Uninsured ) 
               ]
    

    fig = select_ML(df_sel,'lda')
#     fig.update_yaxes(
#     scaleanchor = "x",
#     scaleratio = 1,
#       )
    
    return fig
@app.callback(
    Output('graph-with-slider3', 'figure'),
    Input('year-slider3', 'value'),
    Input('income_level_of_family_income3', 'value'),
    Input('education3', 'value'),
    Input('age_Group_of_Uninsured3', 'value'))
def update_figure3(selected_year,income_level_of_family_income,education,age_Group_of_Uninsured):
    
   
    df_sel = df[(df['TimeFrame'] == int(selected_year)) & (df['Income Level'] == income_level_of_family_income) 
                & (df['Education Level'] == education ) & (df['Age Group'] == age_Group_of_Uninsured ) 
               ]
    

    fig = select_ML(df_sel,'tsne')
#     fig.update_yaxes(
#     scaleanchor = "x",
#     scaleratio = 1,
#       )
    
    return fig
@app.callback(
    Output('graph-with-slider4', 'figure'),
    Input('year-slider4', 'value'),
    Input('my_dropdown', 'value'),
    Input('income_level_of_family_income4', 'value'),
    Input('education4', 'value'),
    Input('age_Group_of_Uninsured4', 'value')
    
     
    
)
   
def update_figure4(selected_year,my_dropdown,income_level_of_family_income,education,age_Group_of_Uninsured):
    
   
    df_sel = map_df[(map_df['TimeFrame'] == int(selected_year)) 
                    & (map_df['Income Level'] == income_level_of_family_income) 
                    & (map_df['Education Level'] == education ) 
                    & (map_df['Age Group'] == age_Group_of_Uninsured )]
    
#     df_sel = Unemployment[df_se['TimeFrame'] == int(selected_year)]

    r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
    r = r.json()
    fig = px.choropleth_mapbox(df_sel,
                               geojson=r,
                               locations="Boro_CD",
                               featureidkey="properties.BoroCD",
                               color= my_dropdown,
                               color_continuous_scale="viridis",
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.7, "lon": -73.9},
                               opacity=0.7,
                               hover_name="Location",
                             
                               height=600
                               )

    
    return fig
@app.callback(
    Output('graph-with-slider5', 'figure'),
    Input('year-slider5', 'value'),
    Input('my_dropdown1', 'value')
   
    
     
    
)
   
def update_figure5(selected_year,my_dropdown1):
#     Unemployment = remove_boro(Unemployment)
# Poverty = remove_boro(Poverty)
# Uninsured = remove_boro(Uninsured)
# Family_Income = remove_boro(Family_Income)
# Educational_Attainment = remove_boro(Educational_Attainment)
          
    df_sel = df_upe[(df_upe['TimeFrame'] == int(selected_year))]
    
#     df_sel = Unemployment[df_se['TimeFrame'] == int(selected_year)]

    r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
    r = r.json()
    fig = px.choropleth_mapbox(df_sel,
                               geojson=r,
                               locations="Fips",
                               featureidkey="properties.BoroCD",
                               color= my_dropdown1,
                               color_continuous_scale="viridis",
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.7, "lon": -73.9},
                               opacity=0.7,
                               hover_name="Location",
                             
                               height=600
                               )
    
    return fig
@app.callback(
    Output('graph-with-slider6', 'figure'),
    Input('year-slider6', 'value'),
    Input('my_dropdown6', 'value'),
    Input('income_level_of_family_income6', 'value'),
  
    
     
    
)
   
def update_figure6(selected_year,my_dropdown,income_level_of_family_income):
    
   
    df_sel = Family_Income[(Family_Income['TimeFrame'] == int(selected_year)) 
                    & (Family_Income['Income Level'] == income_level_of_family_income)]
    
#     df_sel = Unemployment[df_se['TimeFrame'] == int(selected_year)]

    r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
    r = r.json()
    fig = px.choropleth_mapbox(df_sel,
                               geojson=r,
                               locations="Fips",
                               featureidkey="properties.BoroCD",
                               color= my_dropdown,
                               color_continuous_scale="viridis",
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.7, "lon": -73.9},
                               opacity=0.7,
                               hover_name="Location",
                             
                               height=600
                               )
    
    return fig
@app.callback(
    Output('graph-with-slider7', 'figure'),
    Input('year-slider7', 'value'),
    Input('my_dropdown7', 'value'),
    Input('education7', 'value')
   
    
     
    
)
   
def update_figure7(selected_year,my_dropdown,education):
    
   
    df_sel = Educational_Attainment[(Educational_Attainment['TimeFrame'] == int(selected_year)) 
                    & (Educational_Attainment['Education Level'] == education ) 
                   ]
    
#     df_sel = Unemployment[df_se['TimeFrame'] == int(selected_year)]

    r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
    r = r.json()
    fig = px.choropleth_mapbox(df_sel,
                               geojson=r,
                               locations="Fips",
                               featureidkey="properties.BoroCD",
                               color= my_dropdown,
                               color_continuous_scale="viridis",
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.7, "lon": -73.9},
                               opacity=0.7,
                               hover_name="Location",
                             
                               height=600
                               )
    
    return fig
@app.callback(
    Output('graph-with-slider8', 'figure'),
    Input('year-slider8', 'value'),
    Input('my_dropdown8', 'value'),
    Input('age_Group_of_Uninsured8', 'value')
    
     
    
)
def update_figure8(selected_year,my_dropdown,age_Group_of_Uninsured):
    
   
    df_sel = Uninsured[(Uninsured['TimeFrame'] == int(selected_year)) 
                    & (Uninsured['Age Group'] == age_Group_of_Uninsured )]
    
#     df_sel = Unemployment[df_se['TimeFrame'] == int(selected_year)]

    r = requests.get('https://opendata.arcgis.com/datasets/50d6d4b3976249878fdc793d8494a2a3_0.geojson')
    r = r.json()
    fig = px.choropleth_mapbox(df_sel,
                               geojson=r,
                               locations="Fips",
                               featureidkey="properties.BoroCD",
                               color= my_dropdown,
                               color_continuous_scale="viridis",
                               mapbox_style="carto-positron",
                               zoom=9, center={"lat": 40.7, "lon": -73.9},
                               opacity=0.7,
                               hover_name="Location",
                             
                               height=600
                               )
    
    return fig


if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2021 12:38:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:05] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:05] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:10] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 12:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2021 13:21:55] "POST /_dash-update-component

127.0.0.1 - - [24/May/2021 23:58:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:50] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:50] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:22:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/May/2021 00:23:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [25/Ma

In [ ]:
# decision tree
# svr
# try to make uo some statment over the data
# t-sne
# format the x axis and y axis

In [ ]:
# error on the clolr code or dataset